In [ ]:
import pandas as pd
import numpy as np

In [ ]:
old_rate = pd.read_excel("/content/IR_CHG_MPOFull.xlsx", usecols='A:B,BL', header=2, dtype={'a':'datetime64[utc]', 'b':np.float64, 'bl':np.float64}).iloc[38:170].dropna()
old_rate['Дата'] = old_rate['Дата'].apply(pd.to_datetime)

In [ ]:
old_rate = old_rate.set_index('Дата')

In [ ]:
old_rate = old_rate["Ставка рефинансирования"][old_rate.index <= pd.to_datetime('2013-09-16')]

In [ ]:
old_rate = old_rate.resample('D').ffill()
old_rate

,Ставка рефинансирования
Дата,
2006-06-26,11.50
2006-06-27,11.50
2006-06-28,11.50
2006-06-29,11.50
2006-06-30,11.50
...,...
2013-09-12,8.25
2013-09-13,8.25
2013-09-14,8.25


In [ ]:
old_rate = old_rate[old_rate.index >= pd.to_datetime('2006-07-18')]

In [ ]:
old_rate

,Ставка рефинансирования
Дата,
2006-07-18,11.50
2006-07-19,11.50
2006-07-20,11.50
2006-07-21,11.50
2006-07-22,11.50
...,...
2013-09-12,8.25
2013-09-13,8.25
2013-09-14,8.25


In [ ]:
old_rate = old_rate.drop(old_rate.tail(1).index)
old_rate

,Ставка рефинансирования
Дата,
2006-07-18,11.50
2006-07-19,11.50
2006-07-20,11.50
2006-07-21,11.50
2006-07-22,11.50
...,...
2013-09-11,8.25
2013-09-12,8.25
2013-09-13,8.25


In [ ]:
import requests
url = "https://cbr.ru/DailyInfoWebServ/DailyInfo.asmx"
payload = """<?xml version="1.0" encoding="utf-8"?>
            <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
              <soap:Body>
                <KeyRateXML xmlns="http://web.cbr.ru/">
                  <fromDate>2013-09-16</fromDate>
                  <ToDate>2024-11-03</ToDate>
                </KeyRateXML>
              </soap:Body>
            </soap:Envelope>"""

headers = {
    'Content-Type': 'text/xml; charset=utf-8'
}
response = requests.request("POST", url, headers=headers, data=payload)

In [ ]:
from lxml import etree

In [ ]:
rates = [float(kr[1].text) for kr in etree.fromstring(response.text.encode()).iter('KR')]
timestamps = [pd.to_datetime(kr[0].text,utc=True) for kr in etree.fromstring(response.text.encode()).iter('KR')]

In [ ]:
old_rate

,Ставка рефинансирования
Дата,
2006-07-18,11.50
2006-07-19,11.50
2006-07-20,11.50
2006-07-21,11.50
2006-07-22,11.50
...,...
2013-09-11,8.25
2013-09-12,8.25
2013-09-13,8.25


In [ ]:
rate = pd.concat([old_rate, pd.Series(data=rates, index=pd.to_datetime(timestamps,utc=True)).iloc[::-1].resample('1D').ffill().bfill()], names=['Дата', "Ставка"])
rate

,0
2006-07-18 00:00:00,11.5
2006-07-19 00:00:00,11.5
2006-07-20 00:00:00,11.5
2006-07-21 00:00:00,11.5
2006-07-22 00:00:00,11.5
...,...
2024-10-28 00:00:00+00:00,21.0
2024-10-29 00:00:00+00:00,21.0
2024-10-30 00:00:00+00:00,21.0
2024-10-31 00:00:00+00:00,21.0


In [ ]:
rate.to_csv('rate.csv', header=False)

In [ ]:
first = set(rate.index)

In [ ]:
import requests
def get_val_curs(code):
    url = "https://cbr.ru/DailyInfoWebServ/DailyInfo.asmx"
    payload = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
      <soap12:Body>
        <GetCursDynamic xmlns="http://web.cbr.ru/">
          <FromDate>2006-07-18</FromDate>
          <ToDate>2024-11-03</ToDate>
          <ValutaCode>{code}</ValutaCode>
        </GetCursDynamic>
      </soap12:Body>
    </soap12:Envelope>"""

    headers = {
        'Content-Type': 'text/xml; charset=utf-8'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    tree = etree.fromstring(response.text.encode())
    dates = [pd.to_datetime(i.text) for i in tree.iter('CursDate')]
    curses = [float(i.text) for i in tree.iter('VunitRate')]
    return dates, curses

In [ ]:
codes = ['R01235', 'R01239', 'R01375']
data = [get_val_curs(i) for i in codes]

In [ ]:
df = pd.DataFrame(data=[data[0][1], data[1][1], data[2][1]]).T
df = df.rename(columns={0:"Доллар",1:"Евро",2:"Юань"})
df = df.set_index(pd.to_datetime(data[0][0], utc=True)).resample('D').ffill().bfill()
df.to_csv('currencies.csv')

In [ ]:
df['Ставка'] = rate

In [ ]:
df

,Доллар,Евро,Юань,Ставка
2006-07-17 00:00:00+00:00,26.9277,34.0151,3.36689,NaN
2006-07-18 00:00:00+00:00,26.9277,34.0151,3.36689,NaN
2006-07-19 00:00:00+00:00,27.0194,33.8661,3.37489,NaN
2006-07-20 00:00:00+00:00,27.0554,33.8247,3.38023,NaN
2006-07-21 00:00:00+00:00,26.9674,33.9789,3.37261,NaN
...,...,...,...,...
2024-10-29 00:00:00+00:00,97.2300,105.2229,13.61030,21.0
2024-10-30 00:00:00+00:00,97.3261,105.4375,13.59540,21.0
2024-10-31 00:00:00+00:00,97.0530,105.2211,13.58760,21.0
2024-11-01 00:00:00+00:00,97.0226,105.5231,13.60680,21.0


In [ ]:
len(set(df.index) - first)

2619

In [ ]:
df['Дата'][df['Дата'].apply(type) == str]

,Дата


In [ ]:
df.dtypes

,0
Дата,object
Доллар,object
Евро,object
Юань,object
